In [16]:
identifier = 8236122725091915643

import IMAS
import IJulia

if true
    # if using Redis via docker
    REDIS_HOST = "localhost"
    REDIS_PORT = 55000
    REDIS_PASSWORD = "redispw"
else
    REDIS_HOST = "redis-19689.c281.us-east-1-2.ec2.cloud.redislabs.com"
    REDIS_PORT = 19689
    REDIS_PASSWORD = "xU96yd6dhDgRULYmSY9aFTDJGPh7t3DA"
end

stream="$(identifier)__ip"

"8236122725091915643__ip"

In [17]:
subscription_client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);
IMAS.stream_register_service(stream; client=subscription_client)

Any[]

In [18]:
client = IMAS.Jedis.Client(host=REDIS_HOST, port=REDIS_PORT, password=REDIS_PASSWORD);

In [30]:
function run_controller(;kw...)
    kw = NamedTuple(kw)
    if ismissing(my_controller.outputs, :data)
        IMAS.pid_controller(my_controller, kw.P, kw.I, kw.D)
    end
    return my_controller(kw.setpoint, kw.value, kw.time)
end

my_controller = IMAS.controllers__linear_controller()
my_controller.name = "ip"

k=0
while true
    IJulia.clear_output(true)
    display(k)
    inputs = IMAS.pop!(client, "$(stream)__fuse2ctrl"; timeout=10.0, error_on_timeout=false)
    if inputs === nothing
        println("DONE!")
        break
    end
    control = run_controller(; inputs...)
    IMAS.push!(client, "$(stream)__ctrl2fuse"; control)
    k+=1
end

60

DONE!
